## Prediction of store sales

#### Importing Libraries

In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from src.utils import check_missing_data
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#### Loading Data

In [2]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
df_store = pd.read_csv('../data/store.csv')

C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_7832\267565919.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('../data/train.csv')


#### Data Preprocessing 

##### Handling Missing Values

In [3]:
# checking for missing values
missing_data_df_store = check_missing_data(df_store)
print(missing_data_df_store)

                 Column Name  Missing Values  Percentage Missing
3        CompetitionDistance               3            0.269058
4  CompetitionOpenSinceMonth             354           31.748879
5   CompetitionOpenSinceYear             354           31.748879
7            Promo2SinceWeek             544           48.789238
8            Promo2SinceYear             544           48.789238
9              PromoInterval             544           48.789238


In [4]:
# Handle missing values
df_store['CompetitionDistance'].fillna(df_store['CompetitionDistance'].median(), inplace=True)
df_store['CompetitionOpenSinceMonth'].fillna(0, inplace=True)
df_store['CompetitionOpenSinceYear'].fillna(0, inplace=True)
df_store['Promo2SinceWeek'].fillna(0, inplace=True)
df_store['Promo2SinceYear'].fillna(0, inplace=True)
df_store['PromoInterval'].fillna('', inplace=True)

C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_7832\2325696550.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_store['CompetitionDistance'].fillna(df_store['CompetitionDistance'].median(), inplace=True)
C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_7832\2325696550.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object

In [5]:
# checking for missing values
missing_data_df_store = check_missing_data(df_store)
print(missing_data_df_store)

Success: No missing values.


In [6]:
# Remove leading and trailing whitespaces
df_train['StateHoliday'] = df_train['StateHoliday'].str.strip()

In [7]:
# checking for missing values
missing_data_df_train = check_missing_data(df_train)
print(missing_data_df_train)

    Column Name  Missing Values  Percentage Missing
7  StateHoliday          131072           12.885454


In [8]:
# Handle missing values
df_train['StateHoliday'].fillna(0, inplace=True)

C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_7832\1216057678.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['StateHoliday'].fillna(0, inplace=True)


In [9]:
# checking for missing values
missing_data_df_train = check_missing_data(df_train)
print(missing_data_df_train)

Success: No missing values.


In [10]:
# checking for missing values
missing_data_df_test = check_missing_data(df_test)
print(missing_data_df_test)

  Column Name  Missing Values  Percentage Missing
4        Open              11            0.026772


In [11]:
# Handle missing values
df_test['Open'].fillna(0, inplace=True)

C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_7832\3366596396.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test['Open'].fillna(0, inplace=True)


In [12]:
# checking for missing values
missing_data_df_test = check_missing_data(df_test)
print(missing_data_df_test)

Success: No missing values.


##### Converting Non-Numeric Columns to Numeric

In [13]:
df_store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,


In [14]:
# Convert non-numeric columns to numeric
df_store['StoreType'] = df_store['StoreType'].astype('category').cat.codes
df_store['Assortment'] = df_store['Assortment'].astype('category').cat.codes
df_store['PromoInterval'] = df_store['PromoInterval'].astype('category').cat.codes
df_train['StateHoliday'] = df_train['StateHoliday'].astype('category').cat.codes
df_test['StateHoliday'] = df_test['StateHoliday'].astype('category').cat.codes


In [15]:
df_store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,2,0,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,0,0,570.0,11.0,2007.0,1,13.0,2010.0,2
2,3,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,2
3,4,2,2,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,0,0,29910.0,4.0,2015.0,0,0.0,0.0,0


In [16]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,1,1
1,2,5,2015-07-31,6064,625,1,1,1,1
2,3,5,2015-07-31,8314,821,1,1,1,1
3,4,5,2015-07-31,13995,1498,1,1,1,1
4,5,5,2015-07-31,4822,559,1,1,1,1


In [17]:
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


##### Generating New Features

In [18]:
# Extract features from the date column in the test dataset
df_test['Date'] = pd.to_datetime(df_test['Date'])
df_test['Year'] = df_test['Date'].dt.year
df_test['Month'] = df_test['Date'].dt.month
df_test['Day'] = df_test['Date'].dt.day
df_test['Weekday'] = df_test['Date'].dt.weekday
df_test['Weekend'] = df_test['Weekday'] >= 5

In [19]:
# Convert non-numeric columns to numeric
df_test['Weekend'] = df_test['Weekend'].astype('category').cat.codes
df_test['Weekday'] = df_test['Weekday'].astype('category').cat.codes

In [20]:
# Extract features from the date column in the train dataset
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_train['Year'] = df_train['Date'].dt.year
df_train['Month'] = df_train['Date'].dt.month
df_train['Day'] = df_train['Date'].dt.day
df_train['Weekday'] = df_train['Date'].dt.weekday
df_train['Weekend'] = df_train['Weekday'] >= 5

In [21]:
# Convert non-numeric columns to numeric
df_train['Weekend'] = df_train['Weekend'].astype('category').cat.codes
df_train['Weekday'] = df_train['Weekday'].astype('category').cat.codes

In [22]:
# Merge the store data with train and test data
df_train = df_train.merge(df_store, on='Store', how='left')
df_test = df_test.merge(df_store, on='Store', how='left')

In [23]:
# Drop unnecessary columns
df_train.drop(['Date'], axis=1, inplace=True)
df_test.drop(['Date'], axis=1, inplace=True)

In [24]:
# checking for non-numeric columns
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 22 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Sales                      1017209 non-null  int64  
 3   Customers                  1017209 non-null  int64  
 4   Open                       1017209 non-null  int64  
 5   Promo                      1017209 non-null  int64  
 6   StateHoliday               1017209 non-null  int8   
 7   SchoolHoliday              1017209 non-null  int64  
 8   Year                       1017209 non-null  int32  
 9   Month                      1017209 non-null  int32  
 10  Day                        1017209 non-null  int32  
 11  Weekday                    1017209 non-null  int8   
 12  Weekend                    1017209 non-null  int8   
 13  StoreType   

In [25]:
# checking for non-numeric columns
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Id                         41088 non-null  int64  
 1   Store                      41088 non-null  int64  
 2   DayOfWeek                  41088 non-null  int64  
 3   Open                       41088 non-null  float64
 4   Promo                      41088 non-null  int64  
 5   StateHoliday               41088 non-null  int8   
 6   SchoolHoliday              41088 non-null  int64  
 7   Year                       41088 non-null  int32  
 8   Month                      41088 non-null  int32  
 9   Day                        41088 non-null  int32  
 10  Weekday                    41088 non-null  int8   
 11  Weekend                    41088 non-null  int8   
 12  StoreType                  41088 non-null  int8   
 13  Assortment                 41088 non-null  int

##### Scaling the data

In [26]:
# Separate features and target variable from training data
X_train = df_train.drop(['Sales', 'Customers'], axis=1)
y_train = df_train['Sales']

In [27]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
test_scaled = scaler.transform(df_test.drop(['Id'], axis=1))

In [28]:
X_train_scaled

array([[-1.73164032,  0.50148416,  0.45239852, ..., -0.76009695,
        -1.00112755, -0.90615587],
       [-1.72853385,  0.50148416,  0.45239852, ...,  0.08824991,
         0.99712982,  0.94248316],
       [-1.72542738,  0.50148416,  0.45239852, ...,  0.15350736,
         0.99812398,  0.94248316],
       ...,
       [ 1.72275751, -1.00047591, -2.21044047, ..., -0.76009695,
        -1.00112755, -0.90615587],
       [ 1.72586398, -1.00047591, -2.21044047, ..., -0.76009695,
        -1.00112755, -0.90615587],
       [ 1.72897045, -1.00047591, -2.21044047, ...,  0.67556697,
         0.99911814,  1.86680267]])

In [29]:
test_scaled

array([[-1.73164032e+00,  8.30805534e-04,  4.52398520e-01, ...,
        -7.60096948e-01, -1.00112755e+00, -9.06155869e-01],
       [-1.72542738e+00,  8.30805534e-04,  4.52398520e-01, ...,
         1.53507364e-01,  9.98123980e-01,  9.42483158e-01],
       [-1.71300149e+00,  8.30805534e-04,  4.52398520e-01, ...,
        -7.60096948e-01, -1.00112755e+00, -9.06155869e-01],
       ...,
       [ 1.72275751e+00,  1.00213752e+00,  4.52398520e-01, ...,
        -7.60096948e-01, -1.00112755e+00, -9.06155869e-01],
       [ 1.72586398e+00,  1.00213752e+00,  4.52398520e-01, ...,
        -7.60096948e-01, -1.00112755e+00, -9.06155869e-01],
       [ 1.72897045e+00,  1.00213752e+00,  4.52398520e-01, ...,
         6.75566971e-01,  9.99118138e-01,  1.86680267e+00]])

##### Building Models with sklearn Pipelines

In [30]:
# Split the training data for validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

In [34]:
X_train_split

array([[-6.66120129e-01,  1.50279088e+00, -2.21044047e+00, ...,
         1.53507364e-01,  9.98123980e-01,  9.42483158e-01],
       [-1.39924773e+00,  1.50279088e+00, -2.21044047e+00, ...,
        -7.60096948e-01, -1.00112755e+00, -9.06155869e-01],
       [-8.05911408e-01,  5.01484162e-01,  4.52398520e-01, ...,
        -7.60096948e-01, -1.00112755e+00, -9.06155869e-01],
       ...,
       [-6.07097144e-01,  1.00213752e+00,  4.52398520e-01, ...,
        -7.60096948e-01, -1.00112755e+00, -9.06155869e-01],
       [ 4.73955414e-01,  8.30805534e-04,  4.52398520e-01, ...,
        -4.33809693e-01,  9.97129822e-01,  9.42483158e-01],
       [-4.17602299e-01, -1.50112927e+00,  4.52398520e-01, ...,
         1.85020109e+00,  9.98123980e-01,  9.42483158e-01]])

In [37]:
y_train_split

417683       0
659402       0
739873    6244
109671    4055
422541    9459
          ... 
259178       0
365838    7820
131932    8348
671155    7968
121958    9615
Name: Sales, Length: 813767, dtype: int64

In [35]:
X_val_split

array([[ 0.17884049,  1.50279088, -2.21044047, ...,  1.52391383,
         0.99712982,  1.86680267],
       [ 0.10428514, -1.00047591,  0.45239852, ..., -0.76009695,
        -1.00112755, -0.90615587],
       [-0.10074207,  1.00213752,  0.45239852, ...,  2.37226069,
         0.99812398,  1.86680267],
       ...,
       [ 1.5860727 ,  1.00213752,  0.45239852, ...,  0.15350736,
         0.99812398,  0.94248316],
       [-0.73135606,  1.50279088, -2.21044047, ..., -0.43380969,
         1.0001123 ,  0.01816364],
       [ 0.43357127,  1.00213752,  0.45239852, ..., -0.76009695,
        -1.00112755, -0.90615587]])

In [38]:
y_val_split

76435         0
923026     5548
731180     7467
790350     3360
252134    11414
          ...  
574924     8822
752038    11716
38978      3375
193217        0
260336    15388
Name: Sales, Length: 203442, dtype: int64

In [31]:
# Create a pipeline with a Random Forest Regressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [32]:
# Train the model
pipeline.fit(X_train_split, y_train_split)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', RandomForestRegressor(random_state=42))])

In [39]:
# Validate the model
y_val_pred = pipeline.predict(X_val_split)
print(f'Validation RMSE: {np.sqrt(mean_squared_error(y_val_split, y_val_pred))}')

Validation RMSE: 834.734715484004
